# Estimating ECM Parameters from Multi-Pulse HPPC Data

This notebook provides example usage for estimating stationary parameters for a two RC branch Thevenin model using multi-pulse HPPC data.

### Setting up the Environment

Before we begin, we need to ensure that we have all the necessary tools. We will install PyBOP from its development branch and upgrade some dependencies:

In [1]:
%pip install --upgrade pip ipywidgets openpyxl pandas -q
%pip install pybop -q

import pandas as pd
import plotly.graph_objects as go
import pybamm

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In this example, we use the default parameter value for the "Open-circuit voltage [V] as provided by the original PyBaMM class. The other relevant parameters for the ECM model implementation are updated as per the cell specification.

In [2]:
# Load the parameters
parameter_set =  pybamm.equivalent_circuit.Thevenin().default_parameter_values
parameter_set.update(
    {
        "Cell capacity [A.h]": 3,
        "Nominal cell capacity [A.h]": 3,
        "Element-1 initial overpotential [V]": 0,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 2.5,
        "R0 [Ohm]": 1e-3,
        "R1 [Ohm]": 3e-3,
        "C1 [F]": 5e2,
        "Open-circuit voltage [V]":  pybamm.equivalent_circuit.Thevenin().default_parameter_values[
            "Open-circuit voltage [V]"
        ],
    }
)
# Optional arguments - only needed for two RC pairs
parameter_set.update(
    {
        "R2 [Ohm]": 2e-3,
        "C2 [F]": 3e4,
        "Element-2 initial overpotential [V]": 0,
    },
    check_already_exists=False,
)

## Identifying the Parameters

Now that the initial parameter set is constructed, we can start the PyBOP fitting process. First, we define the model class with two RC elements. One important thing here to note is "maximum solver timestep" (dt_max) needs to be set correctly to have a good fit.

In [3]:
model = pybamm.equivalent_circuit.Thevenin(options={"number of rc elements": 2})

We use multiple HPPC pulses from the dataset: Kollmeyer, Phillip; Skells, Michael (2020), “Samsung INR21700 30T 3Ah Li-ion Battery Data”, Mendeley Data, V1, doi: 10.17632/9xyvy2njj3.1 

Initial state can be either initial SoC or initial open-circuit voltage. In this example, we get the initial OCV by accessing the voltage data.

In [4]:
file_loc = r"../../data/Samsung_INR21700/multipulse_hppc.xlsx"
df = pd.read_excel(file_loc, index_col=None, na_values=["NA"])
df = df.drop_duplicates(subset=["Time"], keep="first")

dataset = pybop.Dataset(
    {
        "Time [s]": df["Time"].to_numpy(),
        "Current function [A]": df["Current"].to_numpy(),
        "Voltage [V]": df["Voltage"].to_numpy(),
    }
)
parameter_set.set_initial_state(f"{df['Voltage'].to_numpy()[0]} V")

{'Ambient temperature [K]': 298.15,
 'Boltzmann constant [J.K-1]': 1.380649e-23,
 'C1 [F]': 500.0,
 'C2 [F]': 30000.0,
 'Cell capacity [A.h]': 3,
 'Cell thermal mass [J/K]': 1000,
 'Cell-jig heat transfer coefficient [W/K]': 10,
 'Current function [A]': 100,
 'Electron charge [C]': 1.602176634e-19,
 'Element-1 initial overpotential [V]': 0,
 'Element-2 initial overpotential [V]': 0,
 'Entropic change [V/K]': <function dUdT at 0x79eb7b833420>,
 'Faraday constant [C.mol-1]': 96485.33212331001,
 'Ideal gas constant [J.K-1.mol-1]': 8.31446261815324,
 'Initial SoC': 0.9997173700417682,
 'Initial temperature [K]': 298.15,
 'Jig thermal mass [J/K]': 500,
 'Jig-air heat transfer coefficient [W/K]': 10,
 'Lower voltage cut-off [V]': 2.5,
 'Nominal cell capacity [A.h]': 3,
 'Open-circuit voltage [V]': <function ocv at 0x79eb7b7fa020>,
 'R0 [Ohm]': 0.001,
 'R1 [Ohm]': 0.003,
 'R2 [Ohm]': 0.002,
 'RCR lookup limit [A]': 340,
 'Upper voltage cut-off [V]': 4.2}

Next, we need to define the parameter for identification. The initial guess for the resistance parameter is generated from a random sample of the prior distributions. These are influenced by the `r_guess` parameter below.

In [5]:
r0_guess = 0.005
parameters = [
    pybop.Parameter(
        "R0 [Ohm]",
        prior=pybop.Gaussian(r0_guess, r0_guess / 10),
        bounds=[0, 0.1],
    ),
    pybop.Parameter(
        "R1 [Ohm]",
        prior=pybop.Gaussian(r0_guess, r0_guess / 10),
        bounds=[0, 0.1],
    ),
    pybop.Parameter(
        "R2 [Ohm]",
        prior=pybop.Gaussian(r0_guess, r0_guess / 10),
        bounds=[0, 0.1],
    ),
    pybop.Parameter(
        "C1 [F]",
        prior=pybop.Gaussian(500, 100),
        bounds=[100, 1000],
    ),
    pybop.Parameter(
        "C2 [F]",
        prior=pybop.Gaussian(2000, 500),
        bounds=[1000, 10000],
    ),
]

In [6]:
# To see current vs time profile.
fig1 = go.Figure()
# Add a line trace for current vs. time
fig1.add_trace(
    go.Scatter(
        x=df["Time"].to_numpy(),
        y=df["Current"].to_numpy(),
        mode="lines",  # 'lines', 'markers', or 'lines+markers'
        name="Current vs Time",
    )
)

# Customize layout
fig1.update_layout(
    title="Current vs Time",
    xaxis_title="Time (s)",
    yaxis_title="Current (A)",
    template="plotly",  # Use a Plotly template (optional)
)

# Show the plot
fig1.show()

In [7]:
# To see voltage vs time profile.
fig2 = go.Figure()
# Add a line trace for current vs. time
fig2.add_trace(
    go.Scatter(
        x=df["Time"].to_numpy(),
        y=df["Voltage"].to_numpy(),
        mode="lines",  # 'lines', 'markers', or 'lines+markers'
        name="Voltage vs Time",
    )
)

# Customize layout
fig2.update_layout(
    title="Voltage vs Time",
    xaxis_title="Time (s)",
    yaxis_title="Voltage (V)",
    template="plotly",  # Use a Plotly template (optional)
)

# Show the plot
fig2.show()

Construct the problem builder

In [8]:
builder = (
    pybop.builders.Pybamm()
    .set_dataset(dataset)
    .set_simulation(model, parameter_values=parameter_set)
    .add_cost(pybop.costs.pybamm.SumSquaredError("Voltage [V]", "Voltage [V]"))
)
for param in parameters:
    builder.add_parameter(param)
problem = builder.build()

Next, we construct the optimisation class with our algorithm of choice and run it. In this case, we select the CMA-ES method as it provides global optimisation capability. For the sake of reducing the runtime of this example, we limit the maximum iterations to 100; however, feel free to update this value.

In [ ]:
options = pybop.PintsOptions(
    sigma=[1e-3, 1e-3, 1e-3, 10, 10],
    max_unchanged_iterations=20,
    max_iterations=100,
)
optim = pybop.CMAES(problem, options=options)
results = optim.run()

## Plotting and Visualisation

### Convergence and Parameter Trajectories

To assess the optimisation process, we can plot the convergence of the cost function and the trajectories of the parameters:

In [10]:
pybop.plot.convergence(optim)
pybop.plot.parameters(optim);

### Conclusion

This notebook illustrates how to perform parameter estimation for multi-pulse HPPC data, providing insights into the optimisation process through various visualisations.